# Covid-19 Statistics Aachen: Datenabfrage

Abfrage der Daten von der Website der StädteRegion Aachen und Speichern in einer Excel-Datei für die Datenübergabe an den nächsten Schritt, in dem die Daten aufbereitet werden.

## Vorbereitungen
- Benötigte Imports
- Konfiguration aus zentrale .ini-Datei einlesen
- Konfiguration und Instanzierung des Loggers
- Globale Variablen definieren

In [34]:
import pandas as pd
import numpy as np

import re
from datetime import date
from datetime import timedelta
from datetime import date
from datetime import time
from datetime import datetime

import requests
from bs4 import BeautifulSoup

import logging
import configparser

# Konfiguration einlesen
config = configparser.ConfigParser()
config.read('config.ini')

# Konfiguration des Loggings
# - Die Logging-Ausgaben werden in der lokalen Datei covid-19-datenabfrage.log geschrieben
# - Für die Ausgabe wird eine bestimmte Formatierung konfiguriert
fhandler = logging.FileHandler(filename=config['Logging']['LogFileName'], mode='a')

# TODO: Formatierung finalisieren (Tausendstel-Sekunden, Tag des Monats, 1. Zeichen des Levels)
formatter = logging.Formatter('%(asctime)s %(levelname)-1.1s %(name)-20.20s - %(message)s')
fhandler.setFormatter(formatter)

# Instanzierung und Konfigurierung des Loggers
log = logging.getLogger("datenabfrage")
log.addHandler(fhandler)
log.setLevel(logging.DEBUG)

# Für die Zwischenspeicherung des eingelesenen HTML
# - Wenn die Website in der aktuellen Session schon einmal abgefragt wurde, wird das Ergebnis
#   der Abfrage in dieser Variablen gesichert
# - Dies erleichtert die Entwicklung der nachfolgenden Verabeitungsschritte und führt nicht immer wieder zu
#   neuen überflüssigen Abfragen der Website
html_payload = None

## Einlesen der existierenden Excel-Datei
- Datei: Siehe `config.ini`
- Seite: Siehe `config.ini`
- Einzulesende Spalten: 
  - **A**: Datum im Format 'DD.MM.'
  - **B**: Akkumulierte Anzahl der Infektionen für gesamte Städteregion (inkl. Aachen) als Integerzahl
  - **C**: Akkumulierte Anzahl der Infektionen für die Stadt Aachen als Integerzahl
  - **D**: Anzahl neuer Todesfälle durch Covid-19 für gesamte Städteregion (inkl. Aachen) als Integerzahl
  - **E**: Akkumulierte Anzahl der Todesfälle durch Covid-19 für gesamte Städteregion (inkl. Aachen) als Integerzahl 
  - **F**: Akkumulierte Anzahl der Genesenen für gesamte Städteregion (inkl. Aachen) als Integerzahl
- Spaltentypen: Spalte A als Datum interpretieren
- Die erste Zeile (Header) überspringen
- Label der Spalten explizit setzen

In [32]:
try:
    c19_cases = pd.read_excel(config['Rohdaten']['FileName'], 
                              sheet_name=config['Rohdaten']['SheetName'], 
                              index_col=0,
                              parse_dates=[0],
                              skiprows=[1],
                              names=['Uhrzeit', 'Summe', 'Summe Aachen', 'Summe Todesfälle', 'Summe genesen', 'Akute Fälle' ])
except FileNotFoundError as err: 
    log.warning('Error during pd.read_excel(): {0}'.format(err))
    # Leere DataFrame für den Start erzeugen
    cols = [ 'Uhrzeit', 'Summe', 'Summe Aachen', 'Summe Todesfälle', 'Summe genesen', 'Akute Fälle' ]
    c19_cases = pd.DataFrame(columns=cols, index=pd.DatetimeIndex([], name='Datum'))
    
c19_cases

,Uhrzeit,Summe,Summe Aachen,Summe Todesfälle,Summe genesen,Akute Fälle
Datum,,,,,,
2020-03-09,10:00:00,58,20,NaN,NaN,58
2020-03-10,10:00:00,61,21,NaN,6.0,55
2020-03-11,10:00:00,63,22,NaN,NaN,63
2020-03-12,10:00:00,70,25,NaN,NaN,70
2020-03-13,10:00:00,85,34,NaN,27.0,58
2020-03-14,10:00:00,100,40,NaN,28.0,72
2020-03-15,10:00:00,155,75,NaN,NaN,155
2020-03-16,10:00:00,169,85,NaN,33.0,136
2020-03-17,10:00:00,211,100,2.0,NaN,209


## Datenabfrage

### Funktionen für die Datenabfrage und Extraktion der Rohtexte
- `robot_access_allowed()` - Via robots.txt prüfen, ob die Website durch ein Skript abgefragt und verabeitet werden darf
- `gather_html()` - Website abfragen und HTML zurückliefern
- `gather_text()` - Relevante Texte aus dem von der Website geliefertem HTML extrahieren und zurückliefern

*<u>Anmerkungen:</u>  
Die Website der Städteregion Aachen verwendet keine `robots.txt`, damit wäre die Funktion `robot_access_allowed()` eigentlich überflüssig. Hier ist sie nur der Vollständigkeit halber definiert.*

*Leider unterscheidet sich die HTML-Struktur der aktuellen Meldungen von der des Meldungsarchivs so stark, dass schon an dieser Stelle eine Differenzierung aufgrund des Textes und nicht nur der HTML-Struktur erfolgen muss. Daher wurde das Argument `kind` zur Funktion `gather_html()` hinzugefügt.*

In [19]:
default_user_agent = config['Rohdaten']['UserAgent']

def robot_access_allowed(url, user_agent=default_user_agent):
    """Request robots.txt. If exists, parse robots.txt and return True if scraping by this script is allowed."""
    
    log.debug("access_allowed(" + url + ", " + user_agent + ")")
    
    headers = { 'user-agent': user_agent }
    
    # Extract root of the website's URL
    
    # Request robots.txt from root of the website
    
    # If robots.txt exist, parse content of the file
    
    return True

def gather_html(url, user_agent=default_user_agent):
    """Request Website from <url> and return the HTML delivered by the Website as text."""

    log.debug("gather_html(" + url + ", " + user_agent + ")")
    
    # Website abfragen
    headers = { 'user-agent': user_agent }
    page = requests.get(url)
    log.debug("gather_html/page.status_code: %d", page.status_code)
    for key, val in page.headers.items():
        log.debug("gather_html/page.header(%s): %s", key, val)   
    if page.status_code == requests.codes.ALL_OK: # Alles ok
        return page.text
        
    return None

def gather_text(html_text):
    """Extract relevant text content from <html_text> and return dictionary with extracted text for 'Header', 'Abstract', 'Main'."""

    log.debug(f"gather_text(%d chars)", len(html_text))

    records = []
       
    # Relevante Objekte aus geliefertem HTML extrahieren
    # - Header
    # - Abstract (existiert nur für die aktuellen Meldungen, nicht im Meldungsarchiv)
    # - Main
    soup = BeautifulSoup(html_text, 'html.parser')

    divs = soup.select('div.mid-col article > div.textcontent > div')
    for div in divs:
        # Header erkennen: Nun wenn dieser gefunden wird, macht es Sinn, nachfolgend nach 
        # 'Abstract'- und 'Main'-Texten zu suchen
        header = div.select('h2')
        if header:
            header_text = next(header[0].stripped_strings)

            abstract = div.select('div > div > ul > li')
            abstract_text = ''
            for a in abstract:
                for tx in a.stripped_strings:
                    abstract_text += tx

            main = div.select('div > p')
            main_text = ''
            for tx in main[0].stripped_strings:
                main_text += tx

            records.append({'Header': header_text, 'Abstract': abstract_text, 'Main': main_text })
                        
    log.debug("gather_text: %d records extracted from HTML", len(records))
    
    return records

### Funktionen für die Extraktion der Falldaten aus den Rohtexten
In der ersten Version erfolgt die Extraktion der Daten aus den Texten mit Hilfe regulärer Ausdrücke:
- Datum/Uhrzeit der Meldung wird aus dem `Header` extrahiert.
- Die eigentlichen Daten werden zweimal extrahiert: Einmal aus dem `Abstract` und zum zweiten Mal aus dem `Main`-Text. 
- Falls sowohl `Abstract` als auch `Main` eingelesen wurden, werden anschließend die extrahierten Daten miteinander verglichen. Nur wenn sie übereinstimmen, wird der Datensatz übernommen.

Die für das Parsen der eingelesenen Texte verwendeten regulären Ausdrücke versuchen einerseits, auf Nummer sicher zu gehen, um die richtigen Textstellen für das Einlesen der Zahlen zu treffen, und andererseits einige Freiheitsgrade zuzulassen: Leerzeichen und andere "White Spaces" werden von Menschen hin und wieder vergesssen oder mehrfach eingegeben, Textvariationen kommen vor, in denen einzelne Wörter nicht erscheinen oder hinzugefügt werden; und natürlich kommen auch Schreibfehler vor. Die regulären Ausdrücke sind dadurch schon recht unübersichtlich geworden und berücksichtigen dennoch nicht alle Situationen.

In [28]:
class NewsMeta():
    """Value class for meta data of news entry."""
    pass

class CaseFigures():
    """Value class for case figures, extracted from the news entries."""
    pass

def parse_header(header: str) -> NewsMeta:
    """
    Analyse <header> of the news and return date and time of news or None. 
    
    The <header> is expected to have the following format:
    
    'Aktuelle Lage Stadt und StädteRegion Aachen zum Corona-Virus; Montag, 22. Juni, 10:00 Uhr'
    
    Variables weekday name, date, and time will be extracted from header. Year assumed to be 2020.
    
    Args:
    header - extracted raw text of header
    
    Returns:
    None - if no matching header found
    NewsMeta(weekday_name, day_of_month, month_name, hour, minutes, date, time, datetime)
    """
    
    monthnames = [ "Januar", "Februar", "März", "April", "Mai", "Juni", "Juli", "August", "September", "Oktober", "November", "Dezember" ]
    
    log.debug("parse_header(" + header[:40] + "...)")
    
    pattern = r"^Aktuelle Lage Stadt und StädteRegion Aachen zum Corona-Virus;\s*" \
    "(Montag|Dienstag|Mittwoch|Donnerstag|Freitag|Samstag|Sonntag),\s*" \
    "([0-9]{1,2})\.\s*(Januar|Februar|März|April|Mai|Juni|Juli|August|September|Oktober|November|Dezember),\s*" \
    "([0-9]{1,2}):([0-9]{2})\s*Uhr.*$"
        
    match = re.search(pattern, header)
    if match:
        meta = NewsMeta()
        try:
            meta.weekday_name = match.group(1)
            meta.day_of_month = int(match.group(2))
            meta.month_name = match.group(3)
            meta.hour = int(match.group(4))
            meta.minutes = int(match.group(5))
            meta.date = date(2020, monthnames.index(meta.month_name) + 1, meta.day_of_month)
            meta.time = time(meta.hour, meta.minutes)
            meta.datetime = datetime.combine(meta.date, meta.time)
            log.debug(f"parse_header/meta.weekday: %s", meta.weekday_name)        
            log.debug(f"parse_header/meta.day_of_month: %d", meta.day_of_month)        
            log.debug(f"parse_header/meta.month_name: %s", meta.month_name)        
            log.debug(f"parse_header/meta.hour: %d", meta.hour)        
            log.debug(f"parse_header/meta.minutes: %d", meta.minutes)        
            log.debug(f"parse_header/meta.date: %s", str(meta.date))        
            log.debug(f"parse_header/meta.time: %s", str(meta.time))
            log.debug(f"parse_header/meta.datetime: %s", str(meta.datetime))
            return meta
        except:
            log.warning("parse_header() failed parsing news header [1]")
            return None
    else: 
        log.warning("parse_header() failed parsing news header [2]")
        return None

def parse_cases(kind: str, text: str, pattern_total: str, pattern_recovered: str, pattern_deaths: str, 
                pattern_active: str) -> CaseFigures:
    """
    Analyse <abstract> or <main> text of the news and return Covid-19 case numbers included therein. 
  
    Args:
    kind
    text
    pattern_total
    pattern_recovered
    pattern_deaths
    pattern_active
    
    Returns:
    None - if text could not be parsed successfully
    CaseFigures(total, total_AC, recovered, deaths, active)
    """
    
    log.debug(f"parse_cases(%s, %s)", kind, text[:80])

    figures = CaseFigures()
    
    # Summe der Corona-Fälle extrahieren
    match = re.search(pattern_total, text)
    if match:
        try:
            figures.total = int(match.group(1))
            figures.total_AC = int(match.group(2))
            log.debug(f"parse_cases/figures.total: %d", figures.total)        
            log.debug(f"parse_cases/figures.total_AC: %d", figures.total_AC)        
        except:
            log.warning(f"parse_cases(%s) failed [1]", kind)
            return None
    else: 
        log.warning(f"parse_cases(%s) failed [2]", kind)
        return None
    
    # Summe der wieder Genesenen extrahieren
    match = re.search(pattern_recovered, text)
    if match:
        try:
            figures.recovered = int(match.group(1))
            log.debug(f"parse_cases/figures.recovered: %d", figures.recovered)        
        except:
            log.warning(f"parse_cases(%s) failed [3]", kind)
            return None
    else: 
        log.warning(f"parse_cases(%s) failed [4]", kind)
        return None
    
    # Summe der Corona-Toten extrahieren
    match = re.search(pattern_deaths, text)
    if match:
        try:
            figures.deaths = int(match.group(1))
            log.debug(f"parse_cases/figures.deaths: %d", figures.deaths)        
        except:
            log.warning(f"parse_cases(%s) failed [5]", kind)
            return None
    else: 
        log.warning(f"parse_cases(%s) failed [6]", kind)
        return None
    
    # Anzahl der aktiven Infektionen extrahieren
    match = re.search(pattern_active, text)
    if match:
        try:
            figures.active = int(match.group(1))
            log.debug(f"parse_cases/figures.active: %d", figures.active)        
        except:
            log.warning(f"parse_cases(%s) failed [7]", kind)
            return None
    else: 
        log.warning(f"parse_cases(%s) failed [8]: %s", kind, text)
        return None

    return figures

def parse_abstract(abstract: str) -> CaseFigures:
    """
    Analyse <abstract> of the news and return Covid-19 case numbers included in abstract. 

    The relevant area of the <abstract> is expected to have the following format:
    
    'Aktuell 1999 bestätigte Coronafälle in der StädteRegion Aachen (davon 994 in der Stadt Aachen). '
    '1880 ehemals positiv auf das Corona-Virus getestete Personen sind inzwischen wieder gesund. '
    'Bislang 98 Todesfälle. Damit aktuell 21 nachgewiesene Infizierte.'
    
    Values will be extracted from abstract text. 
    
    Args:
    abstract - extracted raw text of abstract
    
    Returns:
    None - if abstract could not be parsed successfully
    CaseFigures(total, total_AC, recovered, deaths, active)
    """
    
    log.debug("parse_abstract(" + abstract[:40] + "...)")
    
    pattern_total = r"^Aktuell\s*([0-9]{1,6})\s*bestätigte\s*Coronafälle\s*in\s*der\s*StädteRegion\s*(?:Aachen)?\s*" + \
    r"[(]davon\s*([0-9]{1,6})\s*in\s*der\s*Stadt\s*Aachen[)].*"
    pattern_recovered = r"([0-9]{1,6})\s*ehemals\s*positiv\s*auf\s*das\s*Corona-Virus\s*getestete\s*Personen\s*sind\s*inzwischen" + \
    r"\s*wieder\s*gesund.*"
    pattern_deaths = r"Bislang\s*([0-9]{1,6})\s*Todesfälle.*"
    pattern_active = r"Damit\s*aktuell\s*([0-9]{1,6})\s*nachgewiesene\s*Infizie.*"
        
    return parse_cases("Abstract", abstract, pattern_total, pattern_recovered, pattern_deaths, pattern_active)

def parse_main(main: str) -> CaseFigures:
    """
    Analyse <main> text of the news and return Covid-19 case numbers included in main text. 
    
    The relevant area of the <abstract> is expected to have the following format:
      
    'Es gibt insgesamt in der StädteRegion [nunmehr] 1997 positive Fälle, davon 992 in der Stadt Aachen'
    '1876 ehemals positiv auf das Corona-Virus getestete Personen sind inzwischen wieder gesund'
    'Die Zahl der gemeldeten Todesfälle liegt [nach wie vor] bei 98'
    'Damit sind aktuell 23 Menschen in der StädteRegion nachgewiesen infiziert'

    Args:
    
    Returns:
    None - if abstract could not be parsed successfully
    CaseFigures(total, total_AC, recovered, deaths, active)
    """
    log.debug("parse_main(" + main[:40] + "...)")

    pattern_total = r"Es\s*gibt\s*insgesamt\s*in\s*der\s*StädteRegion\s*(?:nunmehr)?\s*([0-9]{1,6})\s*positive\s*Fälle,\s*" + \
    r"davon\s*([0-9]{1,6})\s*in\s*der\s*Stadt\s*Aachen"
    pattern_recovered = r"([0-9]{1,6})\s*ehemals\s*positiv\s*auf\s*das\s*Corona-Virus\s*getestete\s*Personen\s*sind\s*inzwischen\s*" + \
    r"wieder\s*gesund"
    pattern_deaths = r"Die\s*Zahl\s*der\s*gemeldeten\s*Todesfälle\s*liegt\s*(?:nach\s*wie\s*vor)?\s*bei\s*([0-9]{1,6})"
    pattern_active = r"Damit\s*sind\s*aktuell\s*([0-9]{1,6})\s*Menschen\s*in\s*der\s*StädteRegion\s*(?:Aachen)?\s*nachgewiesen\s*infiziert"
    
    return parse_cases("Main", main, pattern_total, pattern_recovered, pattern_deaths, pattern_active)

def checked_data(meta: str, fig_abstract: str, fig_main: str) -> []:
    """
    Check extracted data for consistency and return valid figures or None.
    
    Args:
    
    Returns:
    
    """
    if meta != None: # Der Header konnte erfolgreich analysiert werden
        a, m = fig_abstract, fig_main
        if (a != None) and (m != None): # Sowohl Abstract als auch Main konnten erfolgreich analysiert werden
            if (m.total == a.total) and (m.deaths == a.deaths) and (m.recovered == a.recovered) and \
               (m.active == a.active):
                if m.active == 0:
                    m.active = m.total - m.deaths - m.recovered
                if m.total - m.deaths - m.recovered == m.active:
                    return [ meta.date, meta.time, m.total, m.total_AC, m.deaths, m.recovered, m.active ]
        elif m != None: # Mindestens Main konnte erfolgreich analysiert werden
            if m.active == 0:
                m.active = m.total - m.deaths - m.recovered
            if m.total - m.deaths - m.recovered == m.active:
                return [ meta.date, meta.time, m.total, m.total_AC, m.deaths, m.recovered, m.active ]
    return None

def fill_dataframe_from_text(text_records: str) -> pd.DataFrame:
    """
    Analyse <text_records> and return Pandas dataframe with parsed Covid-19 case numbers. 
    
    Args:
    text_records
    
    Returns:
    pd.DataFrame - 
    """
    log.debug(f"fill_dataframe_from_text(%d records)", len(text_records))

    fig = []
    date = []
    
    for tr in text_records:
        meta = parse_header(tr['Header'])
        fig_abstract = parse_abstract(tr['Abstract'])
        fig_main = parse_main(tr['Main'])
        
        rec = checked_data(meta, fig_abstract, fig_main)
        if rec:
            date.append(rec[0])
            fig.append(rec[1:])

    if fig.count:
        cols = [ 'Uhrzeit', 'Summe', 'Summe Aachen', 'Summe Todesfälle', 'Summe genesen', 'Akute Fälle' ]
        df = pd.DataFrame(np.array(fig), columns=cols, index=pd.DatetimeIndex(date, name='Datum')).sort_index()
        return df
    
    return pd.DataFrame() # Leer

### Datenabfrage durchführen
1. Datum des letzten Datensatzes in der Excel-Datei ermitteln und mit aktuellem Datum vergleichen. 
2. Wenn mindestens 1 Tag seit der letzten Datenabfrage vergangen ist, neue Abfrage durchführen.
3. Von der Website gelieferte Rohtexte auswerten und Fallzahlen extrahieren
4. Extrahierte Fallzahlen auf Konsistenz prüfen
5. Geprüfte Fallzahlen zu neuem DataFrame hinzufügen
6. Den neuen mit dem existierendem DataFrame zusammenführen
7. Die zusammengeführten Daten speichern

In [35]:
new_request = (c19_cases.size == 0) or (date.today() >= (date.fromtimestamp(c19_cases.index.max().timestamp()) + timedelta(days=1)))

url = config['Rohdaten']['SourceURLDefault']
if new_request:
    log.info("New request initiated")
    if not html_payload:
        if robot_access_allowed(url):
            html_payload = gather_html(url)

    if html_payload:
        records = gather_text(html_payload)

    if records.count:
        new_cases = fill_dataframe_from_text(records)

    if not new_cases.empty:
        # Nur Zeilen mit neuerem Datum hinzufügen
        merged_cases = pd.concat([c19_cases, new_cases[new_cases.index > c19_cases.index[-1]]], join='outer')

    merged_cases.to_excel(config['Rohdaten']['FileName'], 
                          sheet_name=config['Rohdaten']['SheetName'], index_label='Datum')

    merged_cases
else:
    log.info("No new request required")
